In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [77]:
# import os
# del os.environ['HF_TOKEN']
# os.environ.get("HF_TOKEN")

In [78]:
from autodm.llm import get_llm, Gemini
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.locaitons import Location, LocationGraph, setup_new_locations
from autodm.adventurelog import AdventureLog
from autodm.storyline import StoryLineWithStoryItems

from typing import List, Union
from rich import print

In [79]:
locations = setup_new_locations()

In [80]:
region = locations[locations.list_locations('region').pop()]
city = locations[locations.get_children(region.name).pop()]

In [81]:
character = Character.generate(level=0)
print(character)

Character(
    chr_class='fighter',
    chr_race='human',
    name='Gorthok the Unyielding',
    attributes=Attributes(strength=19, dexterity=2, constitution=15, intelligence=2, wisdom=6, charisma=16),
    level=0,
    max_hp=25,
    hp=25,
    spells=[],
    equipment=[]
)

In [82]:
# story = StoryLineWithStoryItems.generate(character, city, region, locations)

In [83]:
# print(story)

In [84]:
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core import PromptTemplate
from pydantic import BaseModel, Field
import typing as T
import gradio as gr

In [85]:
log = AdventureLog()

opening = f"""\
You are a {character.chr_race} {character.chr_class} named {character.name} find themself in a city called {city.name}, {city.description}.\
"""
print(opening)

log.add_entry(opening)

adventure_log_tool = QueryEngineTool.from_defaults(
    query_engine=log.index.as_query_engine(llm=get_llm()), 
    description="Use this tool when you want to look up anything that has happened in the game.",
    name='adventure_log_tool'
)

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


You are a human fighter named Gorthok the Unyielding find themself in a city called Wysteria City, The capital city
of the Wysteria Kingdom, known for its grand architecture and bustling streets..

In [86]:
def roll(dice_type: Union[int, DiceType] = DiceType.D20, dice_count: int = 1) -> str:
    "Rolls 1 or more die. Dice can be of type D4, D6, D8, D10, D12, or D20. \
You can also roll multiple dice at once, and the result is the sum of the rolls."
    dice = Dice(type=DiceType(dice_type), count=dice_count)
    return f"Rolling {dice_count}d{dice_type}. Result: {dice.roll()}"

roll_tool = FunctionTool.from_defaults(roll, description="Use this tool when you want to roll dice to determine the outcome of an action.")

In [97]:
agent = ReActAgent.from_tools(tools=[adventure_log_tool, roll_tool], llm=get_llm(), verbose=0)
pt = agent.get_prompts()['agent_worker:system_prompt']
character_str = f"The player's character is a {character.chr_race} {character.chr_class} of level {character.level} named {character.name}. \
The character has {character.hp} out of {character.max_hp} HP, and the following attributes: {character.attributes}. "

new_template = """\
You are a talented D&D dungeon master whose goal is to walk the player through a D&D campaign. \
When the player asks you a question, you should use the tools at your disposal to answer the question. \
If the player takes an action, make sure the result is in line with D&D rules and preveious events in the adventure log. \
If there is no tool available to answer the question, you create exciting and funny experiences for the player. \
Try to match the player's tone and style to keep them engaged. \
You may also roll dice for the player when appropriate, including attack rolls, saving throws, and skill checks. \
%s \

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand. \
This may require breaking the task into subtasks and using different tools to complete each subtask. \
Once you complete the task, you will provide the answer to the player. \
Do not take additional actions on behalf of the player unless instructed to do so - for example, if they say "look around," describe what they see according to the detail of a perception roll. \
Do not take any additional steps. \

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in English using the following format:

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information to answer the question without using 
any more tools. At that point, you MUST respond in the one of the following two formats:

```
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 
```

```
Thought: I cannot answer the question with the provided tools. I will create an engaging experience for the player.
Answer: 
```

## Current Conversation

Below is the current conversation consisting of interleaving human and assistant messages.\
""" % character_str

pt.template = new_template
agent.update_prompts({'agent_worker:system_prompt': pt})

In [101]:
agent.chat('hello')

NotImplementedError: Messages passed in must be of odd length.

In [95]:
def msg(msg: str, user='player') -> str:
    log.add_entry(msg, user=user)
    response = agent.chat(msg)
    log.add_entry(response.response, user='dm')
    return response.response

In [96]:
resp = msg("This is the beggining of the game. Set the scene for a D&D campaign by summarizing the current location. Once you have the location description, you are done.", 'dm')

NotImplementedError: Messages passed in must be of odd length.

In [71]:
import gradio as gr

In [72]:
def respond(message, chat_history):
    bot_message = msg(message)
    chat_history.append((message, bot_message))
    return '', chat_history

In [73]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(value=[("", resp)])
    message = gr.Textbox()
    clear = gr.ClearButton([message, chatbot])

    message.submit(respond, [message, chatbot], [message, chatbot])

demo.launch(inline=False)

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/gradio/queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/gradio/blocks.py", line 1514, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/sit